<a href="https://colab.research.google.com/github/NicolayC/Rugby/blob/main/Rugby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalación y configuración inicial

In [ ]:
# Instalación de dependencias
!pip install torch torchvision ultralytics mlflow
# Mostrar si hay GPU disponible
!nvidia-smi

In [ ]:
!pip install --upgrade mlflow ultralytics

# Dataset

In [ ]:
# Desde Roboflow
!curl -L "InsertarURL" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Importaciones globales

In [ ]:
from ultralytics import YOLO, RTDETR
import mlflow
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Función para inicializar MLflow + entrenamiento

In [ ]:
def entrenar_modelo(model_class, weights_path, experiment_name, run_name, data_yaml, **train_args):
  mlflow.set_experiment(experiment_name)
  mlflow.autolog()

  with mlflow.start_run(run_name=run_name) as run:
    model = model_class(weights_path)
    results = model.train(data=data_yaml, **train_args)
    print(f"Run ID: {run.info.run_id}")

  mlflow.end_run()
  return model

# Entrenamiento con YOLOv8

In [ ]:
yolo_model = entrenar_modelo(
    model_class=YOLO,
    weights_path="yolov8n.pt",
    experiment_name="YOLOv8_Rugby_Test1",
    run_name="training_datav1",
    data_yaml='/content/data.yaml',
    epochs=6,
    # Para ejecutar con GPU
    #device='cuda'
    device='cpu',
    batch=8,
    cos_lr=True,
    augment=True
)

$# Explorador de Archivos

In [ ]:
# Carpetas
!ls /content/runs/detect/predict

# Drive
# !ls /content/drive/MyDrive

# Descargar los pesos

In [ ]:
from google.colab import files
files.download('/content/runs/detect/train2/weights/best.pt')


# Predicción sobre imágenes

In [ ]:
model = YOLO("/content/runs/detect/train2/weights/best.pt")
results = model.predict(source="/content/test/images", conf=0.5, save=True)

# Visualización de una imagen con Matplotlib

In [ ]:
img_path = "/content/runs/detect/predict/frame_0005_jpg.rf.101eb2125b80847890294f2da6a0ea85.jpg"


img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
results = model.predict("/content/test/images/frame_0005_jpg.rf.101eb2125b80847890294f2da6a0ea85.jpg")[0]
annotated = results.plot()

# Mostrar en Colab
import matplotlib.pyplot as plt
import cv2
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Validación del modelo

In [ ]:
model = YOLO("/content/runs/detect/train2/weights/best.pt")
results = model.val(data="data.yaml", split="test")
print(results)

# Predicción sobre video

In [ ]:
video_path = "/content/drive/MyDrive/wheelchair.mp4"
output_path = "/content/drive/MyDrive/output.mp4"

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# model = YOLO("/content/runs/detect/train2/weights/best.pt")
model = YOLO("/content/drive/MyDrive/best.pt")


while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break

  results = model.predict(frame, conf=0.5)[0]

  for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    #conf = float(box.conf[0])
    #cls = int(box.cls[0])
    #label = f"{model.names[cls]} {conf:.2f}"
    conf = box.conf[0].item()
    class_id = int(box.cls[0])
    class_name = model.names[class_id]

    label = f"{class_name} {conf:.2f}"

    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

  out.write(frame)

cap.release()
out.release()
#cv2.destroyAllWindows()